In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
from PIL import Image
import os
import glob

In [2]:
df = pd.read_csv("./city-list-count-10km-img-tz.csv")

In [3]:
DIR = "/Users/damoncrockett/Desktop/huepeak/plots/"

In [4]:
vislist = glob.glob(os.path.join(DIR,"*.png"))

In [5]:
d = pd.DataFrame()

In [6]:
d['local_path'] = vislist
d['Center'] = [os.path.basename(item).split("_")[0] for item in vislist]
d['year'] = [os.path.basename(item).split("_")[1] for item in vislist]
d['total_slices'] = [os.path.basename(item).split("_")[2][:-4] for item in vislist]

In [7]:
d.total_slices = d.total_slices.apply(int)

In [8]:
d12 = d[d.year=='2012']
d14 = d[d.year=='2014']

In [9]:
df.set_index("Center",inplace=True)

In [10]:
d12 = d12.join(df,on="Center")
d14 = d14.join(df,on="Center")

# Scatter

In [11]:
d = d.join(df,on='Center')

In [12]:
print d.lon.min(),d.lat.min(),d.lon.max(),d.lat.max()

-118.2436849 -34.9011127 151.2069902 60.1698557


In [13]:
latmin,latmax,lonmin,lonmax = -90,90,-180,180

In [14]:
latrange = latmax-latmin
lonrange = lonmax-lonmin

In [15]:
scale = 100

In [16]:
px_w = lonrange * scale
px_h = latrange * scale

In [17]:
"""
for i in d12.index:
    filepath = d12.local_path.loc[i]
    im = Image.open(filepath)
        
    lat = d12.lat.loc[i]
    lon = d12.lon.loc[i]
    
    xcoord = int(round( (lon + lonmax) * scale ))
    ycoord = int(round( (latmax - lat) * scale ))
    
    canvas.paste(im,(xcoord,ycoord))

canvas.save('/Users/damoncrockett/Desktop/tmp.png')
"""

"\nfor i in d12.index:\n    filepath = d12.local_path.loc[i]\n    im = Image.open(filepath)\n        \n    lat = d12.lat.loc[i]\n    lon = d12.lon.loc[i]\n    \n    xcoord = int(round( (lon + lonmax) * scale ))\n    ycoord = int(round( (latmax - lat) * scale ))\n    \n    canvas.paste(im,(xcoord,ycoord))\n\ncanvas.save('/Users/damoncrockett/Desktop/tmp.png')\n"

# Histogram

In [18]:
w = []
h = []
for i in d.index:
    filepath = d.local_path.loc[i]
    im = Image.open(filepath)
    w.append(im.width)
    h.append(im.height)

d['w'] = w
d['h'] = h

In [19]:
print d.w.max(),d.h.max()

1848 1848


In [20]:
d12bins = list(d12.offset.unique())
d12bins.sort()

In [21]:
d14bins = list(d14.offset.unique())
d14bins.sort()

In [22]:
px_w = len(d14bins) * 1848
px_h = d14.groupby("offset").size().max() * 1848 

In [27]:
canvas = Image.new('RGB',(px_w,px_h),(100,100,100))

In [28]:
counter = -1
for d14bin in d14bins:
    counter+=1
    tmp = d14[d14.offset==d14bin]
    tmp.sort_values(by="lat",inplace=True)
    ycoord = px_h - 1848
    xcoord = counter * 1848
    
    for i in range(len(tmp)):
        im = Image.open(tmp.local_path.iloc[i])
        canvas.paste(im,(xcoord,ycoord))
        ycoord = ycoord - 1848

In [29]:
canvas_w = canvas.width
canvas_h = canvas.height
canvas = canvas.resize( ( int(round( canvas_w * .1 )), int(round( canvas_h * .1)) ), Image.ANTIALIAS )

In [30]:
canvas.save("/Users/damoncrockett/Desktop/huehist14.png")